In [1]:
import pandas as pd
import numpy as np
import pywhatkit as wpp
from datetime import datetime, timedelta

#SetOptions para ver los dataframes
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth',False)

In [2]:
def dias_alertas():
    hoy = datetime.now()
    dias_semana = {'lunes': 0, 'martes': 1, 'miércoles': 2, 'jueves': 3, 'viernes': 4, 'sábado': 5, 'domingo': 6}

    # Encuentra el último martes y jueves
    last_tuesday = hoy - timedelta(days=(hoy.weekday() - dias_semana['martes']) % 7)
    last_thursday = hoy - timedelta(days=(hoy.weekday() - dias_semana['jueves']) % 7)

    # Devuelve la fecha más reciente entre el último martes y jueves
    return max(last_tuesday, last_thursday).strftime('%d')

# Llama a la función y muestra el resultado
# print(dias_alertas())


In [3]:
path = 'Data/'

#df_matries = pd.read_excel(path + 'Román - M{:d} Seguimiento.xlsx', sheet_name='Estatus').format(mes)
database = pd.read_excel(path + 'BASE CNCI.xlsx')
ausentes = pd.read_excel(path + 'Alertas tempranas {:2} de Abril 2024.xlsx'.format(dias_alertas()), sheet_name='Ausente-posible reprobado')

#### Limpieza del Database

Dropear columnas innecesarias

In [4]:
#Renombrar el nombre de la columna NOMBRE
database.rename(columns = {'NOMBRE                                                                                                                                      ' : "Nombre", 'MATRICULA' : "Matricula"}, inplace= True)

database = database[['Nombre', 'Matricula', 'CE CANALIZADO', 'CARRERA', 'M4', 'Correo', 'Teléfono', 'Teléfono2', 'Teléfono3', 'CURP', 'Agente ', 'Asesor Académico', 'Asesor Financiero ', 'Asesor Escolar']]

# Filtro Cadereyta 
database = database[database['CE CANALIZADO'] == 'CE CADEREYTA']

#### Limpieza y Filtro de Ausentes

In [5]:
# Adios a los registros. Filtro solo Cadereyta
ausentes = ausentes.drop(columns=['Registro 1', 'Registro 2', 'Registro 3', 'Registro 4', 'Registro 5', 'Subproyecto'])
ausentes = ausentes[ausentes['Centro Enlace'].isin(['CADEREYTA', 'CE CADEREYTA', 'CC CADEREYTA'])]

In [6]:
# Juntar las materias
inactivos_agrup = ausentes.groupby("Matricula")["Grupo"].agg(lambda x: ", ".join(x)).reset_index().rename(columns={'Grupo':'Materias'})
ausentes = pd.merge(ausentes, inactivos_agrup, on = "Matricula", how = "left")
ausentes.shape #Materias Agrupadas en la última colunmna

(81, 11)

In [7]:
# Reacomodo de las materias
columna_extra = ausentes["Materias"]
ausentes.drop(columns = ["Materias", 'Modelo'], inplace=True)
ausentes["Grupo"] = columna_extra
ausentes = ausentes.drop_duplicates(subset='Nombre')
ausentes.shape #sin duplicados 

(49, 9)

In [8]:
imagen = 'img/Cerro.jpeg'
mencion_stat = ''
path2 = 'infodata/'

#Inicializar la infodata
try:
    matric_error = pd.read_csv(path2 + 'matriculas_error.csv')
    matric_yacontactadas = pd.read_csv(path2+ 'matriculas_yacontactadas.csv') 
    
    matric_error = matric_error.dropna().values.flatten().tolist()
    matric_yacontactadas = matric_yacontactadas.dropna().values.flatten().tolist() 
except FileNotFoundError:
    matric_error = pd.DataFrame()
    matric_yacontactadas = pd.DataFrame()

In [9]:
# Merge nuestros df, con el fin de no llamar a dos df en el for.
ausentes = pd.merge(ausentes, database[['Matricula','Teléfono', 'Teléfono2', 'M4']], on = 'Matricula')

#### Ciclo de mensajes

In [10]:
for index, row in ausentes.iterrows():
    alumno = row['Nombre']
    matricula = row['Matricula']
    materias = row['Grupo']
    telefono = row['Teléfono']
    status = row['Estatus']
    
    if not np.isnan(telefono):
        telefono = '+52 ' + str(telefono)
    if matricula in matric_error:
        if not np.isnan(row['Teléfono2']):
            telefono2 = '+52 ' + str(row['Teléfono2'])
            telefono = telefono2
            print('Matricula con error: {}, Alumno: {}'.format(matricula, alumno))
            
    if status == ('Inactivo'):
        mencion_stat = 'veo que aún no has ingresado a tus materias: *' + materias +'*'
    elif status == ('Ausente'):
        mencion_stat = 'veo que ya has ingresado a tus materias: *'+ materias +'* \n ¿Has logrado avanzar con tus tareas?'
    if status == ('Posible Reprobado'):
        mencion_stat == '¿Cómo vas con tus tareas?'
        
    if matricula in matric_yacontactadas:
        print('Matricula antes contactada: {}, Alumno: {} '.format(matricula, alumno))    
    else:
        mensaje = 'Hola, que tal ' + alumno + ' ' + matricula + ', soy _Román Rodríguez_ Tutor Presencial del CNCI, me comunico contigo ya que ' + mencion_stat + ' ¿Hay algo en lo que pueda apoyarte?'
        print('Enviado a: ', matricula, alumno)
        print('\n')
        #wpp.sendwhats_image(telefono, imagen, mensaje, 25, True)
        #print(telefono, matricula, alumno)

Matricula antes contactada: AL056425, Alumno: LILIANA ABIGAIL PONCE GAUNA 
Enviado a:  AL082706 ANGEL ALBERTO HERNANDEZ HERNANDEZ


Enviado a:  AL087176 GRECIA SCARLETT REYES SANCHEZ


Matricula antes contactada: AL089902, Alumno: ALHINA VIRIDIANA RODRIGUEZ SANCHEZ 
Enviado a:  AL090088 JUAN JOSE CEJA AMADOR


Matricula con error: AL103269, Alumno: BRYAN VAZQUEZ OROZCO
Matricula antes contactada: AL103269, Alumno: BRYAN VAZQUEZ OROZCO 
Enviado a:  BNL055698 GUILLERMO JUAREZ GARZA


Matricula antes contactada: BNL070921, Alumno: MARIA YESENIA NUÑEZ TREVIÑO 
Enviado a:  BNL070959 ANGEL YOEL MUNGUIA HERNANDEZ


Enviado a:  BNL073235 CARLOS ARTURO TORRES BRANDT


Enviado a:  BNL076176 SALVADOR GARZA BAEZ


Enviado a:  BNL077061 EDUARDO GRANADOS PAZOS


Enviado a:  BNL077086 RICARDO SANCHEZ MARTINEZ


Enviado a:  BNL078554 CAROLINA LISBETH IBARRA GONZALEZ


Matricula antes contactada: BNL078651, Alumno: KIMBERLY CHANTAL IBARRA GONZALEZ 
Enviado a:  BNL079305 JUAN CARLOS RAMIREZ CALVILLO


E